In [6]:
from math import cos, sin, pi, acos

In [8]:
class Circle:
    def __init__(self, x, y, r):
        self.pos = (x, y)
        self.r = r
    def area(self):
        return pi * self.r**2

In [11]:
outer = Circle(0, 0, 2 / 3**0.5 + 1)
s1 = Circle(cos(0 * pi / 3) * 2 / 3**0.5, sin(0 * pi / 3) * 2 / 3**0.5, 1)
s2 = Circle(cos(2 * pi / 3) * 2 / 3**0.5, sin(2 * pi / 3) * 2 / 3**0.5, 1)
s3 = Circle(cos(4 * pi / 3) * 2 / 3**0.5, sin(4 * pi / 3) * 2 / 3**0.5, 1)
circles = [outer, s1, s2, s3]

Solved a problem that involved finding an incircle, am going to use this here:

In [34]:
# https://en.wikipedia.org/wiki/Descartes%27_theorem#:~:text=In%20geometry%2C%20Descartes'%20theorem%20states,three%20given%2C%20mutually%20tangent%20circles.
# k_i is the curvature, k_i = 1 / r_i
def find_internal_curvature(k_1, k_2, k_3):
    first = k_1 + k_2 + k_3
    second = k_1 * k_2 + k_2 * k_3 + k_3 * k_1
    k_ans_1, k_ans_2 = first + 2 * second**0.5, first - 2 * second**0.5
    return k_ans_1 #k_ans_2 # Empirically don't use the second answer. Unsure when it is useful

# https://en.wikipedia.org/wiki/Descartes%27_theorem#:~:text=In%20geometry%2C%20Descartes'%20theorem%20states,three%20given%2C%20mutually%20tangent%20circles.
# k_i is the curvature, k_i = 1 / r_i
# P_1 is a tuple given in cartesian coordinates
def find_internal_circle(k_1, k_2, k_3, P_1, P_2, P_3):
    k_4 = find_internal_curvature(k_1, k_2, k_3)
    r = 1 / k_4
    # Converts cartesian to imaginary (x, y) -> x + iy
    z_1 = P_1[0] + P_1[1] * 1j
    z_2 = P_2[0] + P_2[1] * 1j
    z_3 = P_3[0] + P_3[1] * 1j
    first = z_1 * k_1 + z_2 * k_2 + z_3 * k_3
    second = k_1 * k_2 * z_1 * z_2 + k_2 * k_3 * z_2 * z_3 + k_1 * k_3 * z_1 * z_3
    z_ans_1 = (first + 2 * second**0.5) / k_4
    z_ans_2 = (first - 2 * second**0.5) / k_4
    p_1, p_2 = (z_ans_1.real, z_ans_1.imag), (z_ans_2.real, z_ans_2.imag)
    return p_1, r # Empirically don't use the second answer. Unsure when it is useful

In [35]:
outer.area()

14.585580315313054

In [39]:
MD = 10
def dfs(c1, c2, c3, depth, has_outer=False):
    # If depth is greater than I (10) then break
    if depth >= MD:
        return 0
    k_1 = 1 / c1.r
    k_2 = 1 / c2.r
    k_3 = 1 / c3.r
    if (has_outer):
        k_1 = -k_1
    # Find center circle
    C = find_internal_circle(k_1, k_2, k_3, c1.pos, c2.pos, c3.pos)
    C = Circle(*C[0], C[1])
    # Add area
    T = C.area()
    # Dfs 3 times, replacing center with c_i
    T += dfs(c1, c2, C, depth+1, has_outer=has_outer)
    T += dfs(c1, C, c3, depth+1, has_outer=has_outer)
    T += dfs(C, c2, c3, depth+1, has_outer=False)
    return T

T = 0
T += s1.area() * 3
T += 3 * dfs(outer, s1, s2, 0, True)
T += dfs(s1, s2, s3, 0, False)
print(1 - (T / outer.area()))

# Find area of normal circle
# T := A * 3
# DFS on C_1, C_2, OUTER
# T := T + R * 3
# T := T + DFS(*C_i)
# Should be 1 - T?

0.003960869448080384


In [228]:
def get_coords(c1, c2, rn):
    r1, r2 = c1.r, c2.r
    a = (r1**2+r1*rn+r1*r2-r2*rn)/(r1+r2)
    b = ((r1+rn)**2-a**2)**0.5
    # print(a, b)
    v1 = (r1+r2, 0)
    v2 = (c2.x-c1.x, c2.y-c1.y)
    X = r1+r2
    th = acos(v2[0] / X)
    # print(th / pi * 180)
    d = (r1+rn)
    si = acos(a / d) # + cos(a / d)
    r_th = si + th
    # print(d * cos(r_th), d * sin(r_th))
    return c1.x + d * cos(r_th), c1.y + d * sin(r_th) * (-1 if v2[1] < 0 else 1)

In [230]:
get_coords(s1, s2, 1)

(-0.5773502691896255, -1.0000000000000002)

In [233]:
# c1 will always be outer
def comb_area(c1, c2, c3, d=3):
    if not d:
        return 0
    hasouter = c1.is_outer
    # Pick 2 circles, binary search on radius of last circle. Check if new circle intersects with last circle.
    n_r = 0
    n = 2
    new_c = None
    print(c1.c(), c2.c(), c3.c(), d)
    while n > 0.000001:
        while True:
            t_r = n_r + n
            n_c = get_coords(c2, c3, t_r)
            new_c = Circle(*n_c, t_r)
            if new_c.intersects(c1):
                break
            else:
                n_r = t_r
        n /= 2
    print("New circle is:", new_c.c())

    return new_c.area() + comb_area(c1, c2, new_c, d - 1) + comb_area(c1, c3, new_c, d - 1) + comb_area(c2, c3, new_c, d - 1)

In [235]:
t_area = comb_area(outer, s2, s3) #* 3 + comb_area(s1, s2, s3)

(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-0.5773502691896263, -0.9999999999999999, 1) 3
New circle is: (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125)
(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) 2
New circle is: (-1.780512991034026, 0.7116792707179129, 0.2372264862060547)
(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-1.780512991034026, 0.7116792707179129, 0.2372264862060547) 1
New circle is: (-1.7093378900072596, 1.076606368584791, 0.13457679748535156)
(0, 0, 2.1547005383792515) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) (-1.780512991034026, 0.7116792707179129, 0.2372264862060547) 1
New circle is: (-2.0065415672933575, 0.4696534568003638, 0.09393119812011719)
(-0.5773502691896255, 1.0000000000000002, 1) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) (-1.780512991034026, 0.711

KeyboardInterrupt: 

In [103]:
(t_area + s1.area() + s2.area() + s3.area()) / outer.area()

0.6461709275204175